In [1]:
pip install langchain langchain-community langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires req

# Chat Models

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = '<sua chave aqui>'

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature = 0.3)

In [10]:
llm.invoke([{
     "role": "user",
     "content": "Oi, gemini!"
     }])

AIMessage(content='Oi! Sou eu, o Gemini. Tudo bem por aí? Como posso ajudar? 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--5413a586-9605-431a-91fa-0a274afab9b8-0', usage_metadata={'input_tokens': 6, 'output_tokens': 295, 'total_tokens': 301, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 277}})

In [11]:
resposta = llm.invoke([{
     "role": "user",
     "content": "Oi, gemini!"
     }])

In [12]:
resposta.content

'Oi! Sou o Gemini. Como posso ajudar hoje?'

In [18]:
import asyncio
async def stream_response():
    async for chunk in llm.astream("O que é uma LLM?"):
        print(chunk, end="")

await stream_response()

content='Uma **LLM** significa **Large Language Model** (Modelo de Linguagem Grande).\n\nÉ um tipo de modelo de inteligência artificial (IA), especificamente dentro da área de aprendizado de máquina (Machine Learning), projetado para **ent' additional_kwargs={} response_metadata={'safety_ratings': []} id='run--a4f70ee6-6e71-408e-8571-fb3b55faf355' usage_metadata={'input_tokens': 8, 'output_tokens': 1980, 'total_tokens': 1988, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1930}}content='ender, gerar e interagir com a linguagem humana** de forma muito sofisticada.\n\nVamos quebrar o significado de cada parte:\n\n1.  **Grande (Large):**\n    *   Refere-se ao tamanho massivo' additional_kwargs={} response_metadata={'safety_ratings': []} id='run--a4f70ee6-6e71-408e-8571-fb3b55faf355' usage_metadata={'input_token_details': {'cache_read': 0}, 'output_tokens': 49, 'input_tokens': 0, 'output_token_details': {'reasoning': 0}, 'total_tokens': 49}content=' dos dad

# Prompt Templates

In [19]:
from langchain_core.prompts import PromptTemplate

In [20]:
prompt_template = PromptTemplate.from_template("Me conte uma piada sobre {topic}")

In [22]:
prompt_template.invoke({"topic": "gatos"})

StringPromptValue(text='Me conte uma piada sobre gatos')

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "Você é um assistente prestativo"),
    ("user", "Me conte uma piada sobre {topic}")
])

prompt_template.invoke({"topic": "gatos"})

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente prestativo', additional_kwargs={}, response_metadata={}), HumanMessage(content='Me conte uma piada sobre gatos', additional_kwargs={}, response_metadata={})])

In [24]:
contador_de_piadas = prompt_template | llm

In [25]:
contador_de_piadas.invoke({"topic": "elefantes"})

AIMessage(content='Claro! Aqui vai uma:\n\n**Por que os elefantes não usam computador?**\n\n... Porque eles têm medo do mouse! 😂', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--e0fe14b2-ac49-4a87-acfe-8b1670a91c9f-0', usage_metadata={'input_tokens': 17, 'output_tokens': 148, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 118}})

# Output Parser

In [26]:
from langchain_core.output_parsers import StrOutputParser

In [27]:
chain1 = contador_de_piadas | StrOutputParser()

In [28]:
chain1.invoke({"topic": "fantasmas"})

'Claro! Aqui vai uma:\n\nPor que o fantasma foi ao bar?\n.\n.\n.\nPara pegar uma **bebida espirituosa**! 👻🍹'

Também pode ser usado pra outros formatos:

In [29]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [32]:
class Piada(BaseModel):
    pergunta: str = Field(description="esse campo representa a pergunta para gerar a piada")
    resposta: str = Field(description="esse campo representa a resposta da piada")


parser = JsonOutputParser(pydantic_object=Piada)

In [35]:
prompt = PromptTemplate(
    template="""Crie uma piada sobre o tópico.
    Sua resposta deve ser um json no seguinte formato: {format_instructions}
    Tópico: {topic}
    """,
    input_variables=["topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature = 0)

chain = prompt | llm | parser

chain.invoke({"topic": "computadores"})

{'pergunta': 'Por que o computador foi ao oculista?',
 'resposta': 'Porque ele estava com a tela embaçada!'}

# Chains

In [48]:
prompt_critico_de_piadas =  PromptTemplate.from_template("Piada: {piada}. Responda no formato: Piada: <piada>\n Nota <nota de 0 a 9>\n Justificativa: <explique o porque da nota>")

In [49]:
julgador = contador_de_piadas | prompt_critico_de_piadas | llm

In [47]:
type(julgador)

langchain_core.runnables.base.RunnableSequence

In [50]:
julgador.invoke({"topic": "gatos"})

AIMessage(content='Piada: Aqui vai uma:\n\nPor que os gatos são tão ruins em jogos de cartas?\nPorque eles sempre têm uma **pata** de azar! 😼\nNota 7\nJustificativa: A piada é um trocadilho inteligente e bem construído em português. A expressão "pata de azar" é uma brincadeira com "má sorte" e a palavra "pata" se encaixa perfeitamente no contexto dos gatos. É uma piada leve, divertida e que arranca um sorriso, mesmo que seja um "riso de trocadilho". A execução é clara e o emoji complementa bem.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--5e8a1896-3101-45d7-9cba-77fdf03c76ea-0', usage_metadata={'input_tokens': 236, 'output_tokens': 729, 'total_tokens': 965, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 596}})